# LOB:

Work In Progress: Current Idea is to consturct the LOB in a vectorized fashion where 
- We create a full snapshot of the book for each quote update
- Current approach is to select N rows before a given quote and construct book from here; removing quotes that violate the known Best Bid/Offer 


Current Issues
- Constrain viable spread (benchmark against opposite side of each quote)
- Estimating the Best  Bid on a given exchange
- Factoring in Cancellations to get realistic size vlaues
    - Using Quotes Conditions 

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import collections
from itertools import chain

In [2]:
quotes=pd.read_csv('data/AAPL_quotes.csv')


quotes

,Unnamed: 0,Unnamed: 0.1,Time,Exchange,Symbol,Bid_Price,Bid_Size,Offer_Price,Offer_Size,Best_Bid_Exchange,Best_Bid_Price,Best_Bid_Size,Best_Offer_Exchange,Best_Offer_Price,Best_Offer_Size,date
0,2020-01-06 09:30:00.204637,2020-01-06 09:30:00.204637,2020-01-06 09:30:00.204637,N,AAPL,293.26,1.0,293.78,1.0,K,293.65,1.0,N,293.78,1.0,2020-01-06
1,2020-01-06 09:30:00.204648,2020-01-06 09:30:00.204648,2020-01-06 09:30:00.204648,N,AAPL,293.26,1.0,293.74,1.0,K,293.65,1.0,N,293.74,1.0,2020-01-06
2,2020-01-06 09:30:00.206541,2020-01-06 09:30:00.206541,2020-01-06 09:30:00.206541,N,AAPL,293.26,1.0,294.45,1.0,Q,293.73,1.0,Q,293.78,4.0,2020-01-06
3,2020-01-06 09:30:00.207714,2020-01-06 09:30:00.207714,2020-01-06 09:30:00.207714,N,AAPL,293.26,1.0,293.74,1.0,Q,293.73,1.0,N,293.74,1.0,2020-01-06
4,2020-01-06 09:30:00.209246,2020-01-06 09:30:00.209246,2020-01-06 09:30:00.209246,N,AAPL,293.26,1.0,294.45,1.0,Q,293.73,1.0,Z,293.76,1.0,2020-01-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126698,2020-01-06 15:58:34.719367,2020-01-06 15:58:34.719367,2020-01-06 15:58:34.719367,N,AAPL,299.73,1.0,299.88,1.0,P,299.74,1.0,Q,299.76,6.0,2020-01-06
126699,2020-01-06 15:59:04.031102,2020-01-06 15:59:04.031102,2020-01-06 15:59:04.031102,N,AAPL,299.59,1.0,299.94,1.0,Z,299.78,2.0,P,299.79,1.0,2020-01-06
126700,2020-01-06 15:59:58.768321,2020-01-06 15:59:58.768321,2020-01-06 15:59:58.768321,N,AAPL,299.43,3.0,299.94,1.0,Q,299.74,2.0,Q,299.82,3.0,2020-01-06
126701,2020-01-06 15:59:59.347607,2020-01-06 15:59:59.347607,2020-01-06 15:59:59.347607,N,AAPL,299.43,3.0,299.94,1.0,P,299.74,1.0,Q,299.81,1.0,2020-01-06


In [3]:

quotes.index=quotes['Time']

column_funcs={'Exchange':'last','Symbol':'last','Bid_Price':np.mean,'Bid_Size':np.sum,'Offer_Price':np.mean,'Offer_Size':np.sum,'Best_Bid_Exchange':'last','Best_Bid_Price':'last','Best_Bid_Size':'last','Best_Offer_Exchange':'last','Best_Offer_Price':'last','Best_Offer_Size':'last'}
quotes=quotes.groupby(quotes.index).agg(column_funcs)

# construct LOB at time t via looback period of some duration (i.e. 1000)
quotes['Bid_Price_Nested']=[quotes['Bid_Price'].values[max(i-5000,0):i+1] for i in range(0,quotes.shape[0])]
quotes['Bid_Sizes_Nested']=[quotes['Bid_Size'].values[max(i-5000,0):i+1] for i in range(0,quotes.shape[0])]

 
# use best bid on queried exchange as benchmark for removing stale quotes
quotes['Best_Bid_Price_Exchange']=quotes['Best_Bid_Price'][quotes['Best_Bid_Exchange']==np.unique(quotes['Exchange'].values)[0]]  # benchmark on best bid same exchange
quotes['Best_Bid_Price_Exchange']=quotes['Best_Bid_Price_Exchange'].fillna(np.inf)

#take best bid as the min between highest bid that is less than quoted offer and  bbo across all exchanges 
quotes['Best_Bid_Price_Exchange']=[min(np.max(np.concatenate([quotes['Bid_Price_Nested'].values[i],np.array([0])])),quotes['Best_Bid_Price_Exchange'].values[i],quotes['Offer_Price'].values[i]-0.01) for i in range (0,quotes.shape[0])]
 
# remove stale bids or asks that violate BBO
quotes['Bid_Sizes_Nested']=[quotes['Bid_Sizes_Nested'].values[i][quotes['Bid_Price_Nested'].values[i]<=quotes['Best_Bid_Price_Exchange'].values[i]] for i in range (0,quotes.shape[0])]
quotes['Bid_Price_Nested']=[quotes['Bid_Price_Nested'].values[i][quotes['Bid_Price_Nested'].values[i]<=quotes['Best_Bid_Price_Exchange'].values[i]] for i in range (0,quotes.shape[0])]


 
### reformulate into dictionary 
quotes['Bid_Book']=[dict(zip(quotes['Bid_Price_Nested'].values[i],quotes['Bid_Sizes_Nested'].values[i])) for i in range (0,quotes.shape[0])]


# sort bid prices and construct LOB up to lvl n 
quotes['Bid_Book_n']=[list(chain.from_iterable(list(collections.OrderedDict(sorted(i.items(),reverse=True)).items())[-25:])) for i in quotes['Bid_Book'].values]


# construct LOB at time t via looback period of some duration (i.e. 1000)
quotes['Offer_Price_Nested']=[quotes['Offer_Price'].values[max(i-5000,0):i+1] for i in range(0,quotes.shape[0])]
quotes['Offer_Sizes_Nested']=[quotes['Offer_Size'].values[max(i-5000,0):i+1] for i in range(0,quotes.shape[0])]


# use best bid on queried exchange as benchmark for removing stale quotes
quotes['Best_Offer_Price_Exchange']=quotes['Best_Offer_Price'][quotes['Best_Offer_Exchange']==np.unique(quotes['Exchange'].values)[0]] # assumes one exchange
quotes['Best_Offer_Price_Exchange']=quotes['Best_Offer_Price_Exchange'].fillna(-np.inf)

#take offer as lowest ask on our exchange greater than quoted bid and bbo across all exchanges 
quotes['Best_Offer_Price_Exchange']=[max(np.min(np.concatenate([quotes['Offer_Price_Nested'].values[i],np.array([np.inf])])),quotes['Best_Offer_Price_Exchange'].values[i],quotes['Bid_Price'].values[i]+0.01) for i in range (0,quotes.shape[0])]


# remove stale bids or asks that violate BBO
quotes['Offer_Sizes_Nested']=[quotes['Offer_Sizes_Nested'].values[i][quotes['Offer_Price_Nested'].values[i]>=quotes['Best_Offer_Price_Exchange'].values[i]] for i in range (0,quotes.shape[0])]
quotes['Offer_Price_Nested']=[quotes['Offer_Price_Nested'].values[i][quotes['Offer_Price_Nested'].values[i]>=quotes['Best_Offer_Price_Exchange'].values[i]] for i in range (0,quotes.shape[0])]

### reformulate into dictionary 
quotes['Offer_Book']=[dict(zip(quotes['Offer_Price_Nested'].values[i],quotes['Offer_Sizes_Nested'].values[i])) for i in range (0,quotes.shape[0])]


# sort bid prices and construct LOB up to lvl n 
quotes['Offer_Book_n']=[list(chain.from_iterable(list(collections.OrderedDict(sorted(i.items())).items())[:25])) for i in quotes['Offer_Book'].values]



In [4]:
quotes

,Exchange,Symbol,Bid_Price,Bid_Size,Offer_Price,Offer_Size,Best_Bid_Exchange,Best_Bid_Price,Best_Bid_Size,Best_Offer_Exchange,...,Bid_Price_Nested,Bid_Sizes_Nested,Best_Bid_Price_Exchange,Bid_Book,Bid_Book_n,Offer_Price_Nested,Offer_Sizes_Nested,Best_Offer_Price_Exchange,Offer_Book,Offer_Book_n
Time,,,,,,,,,,,,,,,,,,,,,
2020-01-06 09:30:00.204637,N,AAPL,293.26,1.0,293.78,1.0,K,293.65,1.0,N,...,[293.26],[1.0],293.26,{293.26: 1.0},"[293.26, 1.0]",[293.78],[1.0],293.78,{293.78: 1.0},"[293.78, 1.0]"
2020-01-06 09:30:00.204648,N,AAPL,293.26,1.0,293.74,1.0,K,293.65,1.0,N,...,"[293.26, 293.26]","[1.0, 1.0]",293.26,{293.26: 1.0},"[293.26, 1.0]","[293.78, 293.74]","[1.0, 1.0]",293.74,"{293.78: 1.0, 293.74: 1.0}","[293.74, 1.0, 293.78, 1.0]"
2020-01-06 09:30:00.206541,N,AAPL,293.26,1.0,294.45,1.0,Q,293.73,1.0,Q,...,"[293.26, 293.26, 293.26]","[1.0, 1.0, 1.0]",293.26,{293.26: 1.0},"[293.26, 1.0]","[293.78, 293.74, 294.45]","[1.0, 1.0, 1.0]",293.74,"{293.78: 1.0, 293.74: 1.0, 294.45: 1.0}","[293.74, 1.0, 293.78, 1.0, 294.45, 1.0]"
2020-01-06 09:30:00.207714,N,AAPL,293.26,1.0,293.74,1.0,Q,293.73,1.0,N,...,"[293.26, 293.26, 293.26, 293.26]","[1.0, 1.0, 1.0, 1.0]",293.26,{293.26: 1.0},"[293.26, 1.0]","[293.78, 293.74, 294.45, 293.74]","[1.0, 1.0, 1.0, 1.0]",293.74,"{293.78: 1.0, 293.74: 1.0, 294.45: 1.0}","[293.74, 1.0, 293.78, 1.0, 294.45, 1.0]"
2020-01-06 09:30:00.209246,N,AAPL,293.26,1.0,294.45,1.0,Q,293.73,1.0,Z,...,"[293.26, 293.26, 293.26, 293.26, 293.26]","[1.0, 1.0, 1.0, 1.0, 1.0]",293.26,{293.26: 1.0},"[293.26, 1.0]","[293.78, 293.74, 294.45, 293.74, 294.45]","[1.0, 1.0, 1.0, 1.0, 1.0]",293.74,"{293.78: 1.0, 293.74: 1.0, 294.45: 1.0}","[293.74, 1.0, 293.78, 1.0, 294.45, 1.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-06 15:58:34.719367,N,AAPL,299.73,1.0,299.88,1.0,P,299.74,1.0,Q,...,"[299.7, 299.69, 299.69, 299.69, 299.71, 299.71...","[4.0, 3.0, 2.0, 2.0, 5.0, 1.0, 4.0, 4.0, 4.0, ...",299.78,"{299.7: 2.0, 299.69: 2.0, 299.71: 3.0, 299.68:...","[299.735, 6.0, 299.73, 1.0, 299.72, 4.0, 299.7...","[299.74, 299.74, 299.74, 299.74, 299.74, 299.7...","[3.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, ...",299.74,"{299.74: 2.0, 299.75: 2.0, 299.76: 2.0, 299.84...","[299.74, 2.0, 299.75, 2.0, 299.755, 11.0, 299...."
2020-01-06 15:59:04.031102,N,AAPL,299.59,1.0,299.94,1.0,Z,299.78,2.0,P,...,"[299.69, 299.69, 299.69, 299.71, 299.71, 299.7...","[3.0, 2.0, 2.0, 5.0, 1.0, 4.0, 4.0, 4.0, 2.0, ...",299.78,"{299.69: 2.0, 299.71: 3.0, 299.7: 2.0, 299.68:...","[299.735, 6.0, 299.73, 1.0, 299.72, 4.0, 299.7...","[299.73, 299.71, 299.72, 299.74, 299.73, 299.7...","[2.0, 2.0, 3.0, 3.0, 5.0, 5.0, 5.0, 6.0, 7.0, ...",299.60,"{299.73: 2.0, 299.71: 2.0, 299.72: 4.0, 299.74...","[299.6, 4.0, 299.61, 4.0, 299.62, 4.0, 299.63,..."
2020-01-06 15:59:58.768321,N,AAPL,299.43,3.0,299.94,1.0,Q,299.74,2.0,Q,...,"[299.69, 299.69, 299.71, 299.71, 299.7, 299.7,...","[2.0, 2.0, 5.0, 1.0, 4.0, 4.0, 4.0, 2.0, 2.0, ...",299.78,"{299.69: 2.0, 299.71: 3.0, 299.7: 2.0, 299.68:...","[299.73, 1.0, 299.72, 4.0, 299.71, 3.0, 299.7,...","[299.71, 299.72, 299.74, 299.73, 299.73, 299.7...","[2.0, 3.0, 3.0, 5.0, 5.0, 5.0, 6.0, 7.0, 4.0, ...",299.56,"{299.71: 2.0, 299.72: 4.0, 299.74: 2.0, 299.73...","[299.56, 4.0, 299.57, 6.0, 299.58, 5.0, 299.59..."


In [5]:
quotes.iloc[-1]

Exchange                                                                     N
Symbol                                                                    AAPL
Bid_Price                                                               299.81
Bid_Size                                                                     1
Offer_Price                                                             299.94
Offer_Size                                                                   1
Best_Bid_Exchange                                                            Q
Best_Bid_Price                                                          299.81
Best_Bid_Size                                                                2
Best_Offer_Exchange                                                          Q
Best_Offer_Price                                                        299.86
Best_Offer_Size                                                             25
Bid_Price_Nested             [299.71, 299.71, 299.7,

In [6]:
quotes.iloc[-1]['Offer_Book_n']

[299.83,
 1.0,
 299.84,
 1.0,
 299.85,
 1.0,
 299.88,
 1.0,
 299.9,
 1.0,
 299.91,
 1.0,
 299.92,
 1.0,
 299.94,
 1.0,
 299.99,
 7.0]

In [7]:
bid_df=pd.DataFrame(quotes['Bid_Book_n'])
columns=np.array(list(chain.from_iterable([(f'Bid_Size_{24-i}',f'Bid_Price_{24-i}') for i in range(0,25)])))[::-1]

bid_df=pd.DataFrame(bid_df['Bid_Book_n'].values.tolist(),index=bid_df.index,columns=columns)



bid_df

,Bid_Price_0,Bid_Size_0,Bid_Price_1,Bid_Size_1,Bid_Price_2,Bid_Size_2,Bid_Price_3,Bid_Size_3,Bid_Price_4,Bid_Size_4,...,Bid_Price_20,Bid_Size_20,Bid_Price_21,Bid_Size_21,Bid_Price_22,Bid_Size_22,Bid_Price_23,Bid_Size_23,Bid_Price_24,Bid_Size_24
Time,,,,,,,,,,,,,,,,,,,,,
2020-01-06 09:30:00.204637,293.260,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-06 09:30:00.204648,293.260,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-06 09:30:00.206541,293.260,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-06 09:30:00.207714,293.260,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-06 09:30:00.209246,293.260,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-06 15:58:34.719367,299.735,6.0,299.73,1.0,299.72,4.0,299.71,3.0,299.700,2.0,...,299.55,1.0,299.54,1.0,299.53,2.0,299.46,1.0,299.38,1.0
2020-01-06 15:59:04.031102,299.735,6.0,299.73,1.0,299.72,4.0,299.71,3.0,299.700,2.0,...,299.55,1.0,299.54,1.0,299.53,2.0,299.46,1.0,299.38,1.0
2020-01-06 15:59:58.768321,299.730,1.0,299.72,4.0,299.71,3.0,299.70,2.0,299.695,9.0,...,299.54,1.0,299.53,2.0,299.46,1.0,299.43,3.0,299.38,1.0


In [8]:
offer_df=pd.DataFrame(quotes['Offer_Book_n'])
columns=list(chain.from_iterable([(f'Offer_Price_{i}',f'Offer_Size_{i}') for i in range(0,25)]))

offer_df=pd.DataFrame(offer_df['Offer_Book_n'].values.tolist(),index=offer_df.index,columns=columns) 
 
offer_df

,Offer_Price_0,Offer_Size_0,Offer_Price_1,Offer_Size_1,Offer_Price_2,Offer_Size_2,Offer_Price_3,Offer_Size_3,Offer_Price_4,Offer_Size_4,...,Offer_Price_20,Offer_Size_20,Offer_Price_21,Offer_Size_21,Offer_Price_22,Offer_Size_22,Offer_Price_23,Offer_Size_23,Offer_Price_24,Offer_Size_24
Time,,,,,,,,,,,,,,,,,,,,,
2020-01-06 09:30:00.204637,293.78,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-06 09:30:00.204648,293.74,1.0,293.78,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-06 09:30:00.206541,293.74,1.0,293.78,1.0,294.450,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-06 09:30:00.207714,293.74,1.0,293.78,1.0,294.450,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-06 09:30:00.209246,293.74,1.0,293.78,1.0,294.450,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-06 15:58:34.719367,299.74,2.0,299.75,2.0,299.755,11.0,299.76,2.0,299.77,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-06 15:59:04.031102,299.60,4.0,299.61,4.0,299.620,4.0,299.63,4.0,299.64,1.0,...,299.77,2.0,299.78,2.0,299.790,4.0,299.80,2.0,299.81,2.0
2020-01-06 15:59:58.768321,299.56,4.0,299.57,6.0,299.580,5.0,299.59,5.0,299.60,4.0,...,299.74,2.0,299.75,2.0,299.755,11.0,299.76,2.0,299.77,2.0


In [9]:
lob=pd.concat([bid_df[bid_df.columns[::-1]],offer_df]).sort_index()
lob=lob.ffill().dropna()


#filter after market hours
#lob=lob[lob.index<"2020-01-10 16:00:00"]
#lob=lob[lob.index>"2020-01-06 9:00:00"]

### investigate liquidity of exchange 
 

In [10]:
lob[lob.columns[40:60]][200:250]

,Bid_Size_4,Bid_Price_4,Bid_Size_3,Bid_Price_3,Bid_Size_2,Bid_Price_2,Bid_Size_1,Bid_Price_1,Bid_Size_0,Bid_Price_0,Offer_Price_0,Offer_Size_0,Offer_Price_1,Offer_Size_1,Offer_Price_2,Offer_Size_2,Offer_Price_3,Offer_Size_3,Offer_Price_4,Offer_Size_4
Time,,,,,,,,,,,,,,,,,,,,
2020-01-06 09:30:51.436013,2.0,293.34,2.0,293.35,2.0,293.36,4.0,293.37,2.0,293.38,293.49,2.0,293.52,1.0,293.53,4.0,293.54,4.0,293.55,2.0
2020-01-06 09:30:51.455980,2.0,293.34,2.0,293.35,2.0,293.36,4.0,293.37,2.0,293.38,293.49,2.0,293.52,1.0,293.53,4.0,293.54,4.0,293.55,2.0
2020-01-06 09:30:51.455980,2.0,293.34,2.0,293.35,2.0,293.36,4.0,293.37,2.0,293.38,293.57,2.0,293.58,4.0,293.59,2.0,293.60,2.0,293.61,2.0
2020-01-06 09:30:51.674977,2.0,293.34,2.0,293.35,2.0,293.36,4.0,293.37,2.0,293.38,293.57,2.0,293.58,4.0,293.59,2.0,293.60,2.0,293.61,2.0
2020-01-06 09:30:51.674977,2.0,293.34,2.0,293.35,2.0,293.36,4.0,293.37,2.0,293.38,293.49,2.0,293.52,1.0,293.53,4.0,293.54,4.0,293.55,2.0
2020-01-06 09:30:51.773390,2.0,293.34,2.0,293.35,2.0,293.36,4.0,293.37,2.0,293.38,293.56,4.0,293.57,2.0,293.58,2.0,293.59,2.0,293.60,2.0
2020-01-06 09:30:51.773390,2.0,293.34,2.0,293.35,2.0,293.36,4.0,293.37,2.0,293.38,293.56,4.0,293.57,2.0,293.58,2.0,293.59,2.0,293.60,2.0
2020-01-06 09:30:51.902196,2.0,293.34,2.0,293.35,2.0,293.36,4.0,293.37,2.0,293.38,293.56,4.0,293.57,2.0,293.58,2.0,293.59,2.0,293.60,2.0
2020-01-06 09:30:51.902196,2.0,293.34,2.0,293.35,2.0,293.36,4.0,293.37,2.0,293.38,293.49,2.0,293.52,1.0,293.53,4.0,293.54,4.0,293.55,2.0


### Example LOB Class

In [ ]:
class limit_orderbook():
    '''Class for LOB
    
        Note buy and offer size are expressed in round lots (100 of trade units )
    '''
    
    def __init__(self,quotes,max_level=10,lookback_window=1000):
        self.quotes=quotes
        self.max_level=max_level
        self.lookback_window=lookback_window
        
        self.bids={}
        self.asks={}
        
    def preprocess(self):
        self.quotes.index=self.quotes['Time']
        self.quotes=self.quotes.drop(columns='Time')
        self.quotes=self.quotes.sort_index()
        
        column_funcs={'Exchange':'last','Symbol':'last','Bid_Price':np.mean,'Bid_Size':np.sum,'Offer_Price':np.mean,'Offer_Size':np.sum,'Best_Bid_Exchange':'last','Best_Bid_Price':'last','Best_Bid_Size':'last','Best_Offer_Exchange':'last','Best_Offer_Price':'last','Best_Offer_Size':'last'}
        self.quotes=self.quotes.groupby(self.quotes.index).agg(column_funcs)
    
        return
    
    def walk_book(self):
        
        

        # construct LOB at time t via looback period of some duration (i.e. 1000)
        self.quotes['Bid_Price_Nested']=[self.quotes['Bid_Price'].values[max(i-self.lookback_window,0):i] for i in range(0,self.quotes.shape[0])]
        self.quotes['Bid_Sizes_Nested']=[self.quotes['Bid_Size'].values[max(i-self.lookback_window,0):i] for i in range(0,self.quotes.shape[0])]


        # use best bid on queried exchange as benchmark for removing stale quotes
        self.quotes['Best_Bid_Price_Exchange']=self.quotes['Best_Bid_Price'][self.quotes['Best_Bid_Exchange']==np.unique(self.quotes['Exchange'].values)[0]] # assumes one exchange
        self.quotes['Best_Bid_Price_Exchange']=self.quotes['Best_Bid_Price_Exchange'].ffill()
        self.quotes['Best_Bid_Price_Exchange']=self.quotes['Best_Bid_Price_Exchange'].fillna(np.inf) # still need to adjust this 


        # remove stale bids or asks that violate BBO
        self.quotes['Bid_Sizes_Nested']=[self.quotes['Bid_Sizes_Nested'].values[i][self.quotes['Bid_Price_Nested'].values[i]<self.quotes['Best_Bid_Price_Exchange'].values[i]] for i in range (0,self.quotes.shape[0])]
        self.quotes['Bid_Price_Nested']=[self.quotes['Bid_Price_Nested'].values[i][self.quotes['Bid_Price_Nested'].values[i]<self.quotes['Best_Bid_Price_Exchange'].values[i]] for i in range (0,self.quotes.shape[0])]

        ### reformulate into dictionary 
        self.quotes['Bid_Book']=[dict(zip(self.quotes['Bid_Price_Nested'].values[i],self.quotes['Bid_Sizes_Nested'].values[i])) for i in range (0,self.quotes.shape[0])]


        # sort bid prices and construct LOB up to lvl n 
        self.quotes['Bid_Book_n']=[list(collections.OrderedDict(sorted(i.items())).items())[-self.max_level:] for i in self.quotes['Bid_Book'].values]


        # construct LOB at time t via looback period of some duration (i.e. 1000)
        self.quotes['Offer_Price_Nested']=[self.quotes['Offer_Price'].values[max(i-self.lookback_window,0):i] for i in range(0,self.quotes.shape[0])]
        self.quotes['Offer_Sizes_Nested']=[self.quotes['Offer_Size'].values[max(i-self.lookback_window,0):i] for i in range(0,self.quotes.shape[0])]


        # use best bid on queried exchange as benchmark for removing stale quotes
        self.quotes['Best_Offer_Price_Exchange']=self.quotes['Best_Offer_Price'][self.quotes['Best_Offer_Exchange']==np.unique(self.quotes['Exchange'].values)[0]] # assumes one exchange
        self.quotes['Best_Offer_Price_Exchange']=self.quotes['Best_Offer_Price_Exchange'].ffill()
        self.quotes['Best_Offer_Price_Exchange']=self.quotes['Best_Offer_Price_Exchange'].fillna(np.inf) # still need to adjust this 


        # remove stale bids or asks that violate BBO
        self.quotes['Offer_Sizes_Nested']=[self.quotes['Offer_Sizes_Nested'].values[i][self.quotes['Offer_Price_Nested'].values[i]>self.quotes['Best_Offer_Price_Exchange'].values[i]] for i in range (0,self.quotes.shape[0])]
        self.quotes['Offer_Price_Nested']=[self.quotes['Offer_Price_Nested'].values[i][self.quotes['Offer_Price_Nested'].values[i]>self.quotes['Best_Offer_Price_Exchange'].values[i]] for i in range (0,self.quotes.shape[0])]

        ### reformulate into dictionary 
        self.quotes['Offer_Book']=[dict(zip(self.quotes['Offer_Price_Nested'].values[i],self.quotes['Offer_Sizes_Nested'].values[i])) for i in range (0,quotes.shape[0])]


        # sort bid prices and construct LOB up to lvl n 
        self.quotes['Offer_Book_n']=[list(collections.OrderedDict(sorted(i.items())).items())[:self.max_level] for i in self.quotes['Offer_Book'].values]

        
        
        return 
    